# Retrieval Augmented Generation w/ Watson Discovery & watsonx.ai

This notebook introduces RAG (Retrieval Augmented Generation) by making use of several IBM tools including watsonx.ai to generate answers and garner insights from documents and Watson Discovery as an API to search and answer questions about business documents using custom natural language processing.

This lab should take 30-40 minutes.

Before we begin lets start off by ensuring we have completed some pre-requisites; ensure you gave the following

- IBM Cloud API key 
- Project ID associated with your watsonx instance
- Project ID associated with your Watson Discovery instanace 
- Service URL for Watson Discovery instance

You can use the following support links if you need any help with the pre-requisites above

- [Creating IBM Cloud API Key](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui#create_user_key)
- [Finding watsonx Project ID](https://www.ibm.com/docs/en/watsonx-as-a-service?topic=library-project-id)
- [Finding Watson Discovery Project ID](https://cloud.ibm.com/docs/discovery-data?topic=discovery-data-getting-started)
- [Determining Watson Discovery Service URL](https://cloud.ibm.com/apidocs/discovery-data?code=python#endpoint-cloud)
    - Note this support link shows you the base url, you will need to find the instance url from resource list in the IBM Cloud dashboard.

In [7]:
# Download dependencies

import sys
!{sys.executable} -m pip install -q ibm_watson
!{sys.executable} -m pip install -q ibm_cloud_sdk_core
!{sys.executable} -m pip install -q ibm_watson_machine_learning
!pip install python-dotenv==1.0.0


In [8]:
# Import the necessary packages

from ibm_watson import DiscoveryV2
from ibm_watson.discovery_v2 import QueryLargePassages
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_watson_machine_learning.foundation_models import Model


### Setting up authentication and Watson Discovery

Watson Discovery is a cognitive search and content analytics engine that you can add to applications to identify patterns, 
trends and actionable insights to drive better decision-making. For this lab, we will be making use of Watson Discovery 
to store, analyze, and search the documents which will act as a knowledge base for the question we will eventually be 
asking watsonx.ai.

If you are completing this lab in a workshop setting, a Watson Discovery instance with a collection of documents may 
already be set-up for you; ask your lab instructor. If not, you can create your own Watson Discovery instance from 
IBM Cloud with your own collection of documents, for consistency of this lab you can find a link to the documents used 
for this collections linked [here](https://github.com/ibm-build-lab/VAD-VAR-Workshop/tree/87155f66db7248994ff17fc0dfe80a3b99b64fc9/content/Watsonx/WatsonxAI/docs).

In [10]:
# Your IBM Cloud API key
api_key = "mXMwkYwWPgTl2ORNtFEfhCOlkOFQQt6f92hZITvDSKEy"

# Project ID of your watson discovery collection instance
discovery_project_id = "3f412ff2-5c0a-4cc8-9ae9-568672dcde0e"

# Service URL for your watson discovery instance
service_url = 'https://api.us-south.discovery.watson.cloud.ibm.com/instances/dea4a8e6-09d2-46cc-82d7-3d6820a2ac43'

# Project ID of your watsonx project
watsonx_project_id = "7dff0029-1f9b-47b4-87bc-79d0b4da0820"

# Create IBM IAM authenticator object using your IBM Cloud API Key
authenticator = IAMAuthenticator(api_key)

# Create an Watson Discovery object using the authenticator object previously created
discovery = DiscoveryV2(
    version='2020-08-30',
    authenticator=authenticator
)

# Set the Watson Discovery service url within our discovery object
discovery.set_service_url(service_url)
print("done")


done


In [11]:
# Natural language query for Watson Discovery collection
discovery_question="What is the outstanding long term debt at the end of 2023?"

def query_discovery(question):
  passages = {
    "enabled": True, 
    "per_document": True, 
    "find_answers": True,
    "max_per_document": 1, 
    "characters": 500
   }

  query_large_passages_model = QueryLargePassages.from_dict(passages)

  return discovery.query(
          project_id=discovery_project_id,
          natural_language_query=question,
          passages=query_large_passages_model,
          count=1
      ).get_result()


discovery_json = query_discovery(discovery_question)
print("done")
# print(discovery_json)

done


In [21]:
# Watson Discovery returns a list of documents broken into passages from a given collection
# We want to combine this passages so it can serve as a knowledge base that we provide to watsonx.ai
disc_results = []
combined_disc_results = []
for doc_index in range(len(discovery_json["results"])):
    for j in range(len(discovery_json["results"][doc_index])):
        passages = discovery_json["results"][doc_index]["document_passages"]
        disc_results = []
        for item in passages:
            item = item["passage_text"].replace("<em>","")
            item = item.replace("</em>", "")
            disc_results.append(item)
    combined_disc_results.append("\n".join(disc_results))
print("done")
print("Context to LLM : ")
print(combined_disc_results)

done
Context to LLM : 
['Net Investment Hedge Instruments The Company has designated non-derivative foreign currency denominated long-term debt and the related accrued interest as hedges of its net investment in certain foreign operations. Accordingly, the foreign currency translation of the debt instrument and accrued interest is recorded in accumulated other comprehensive loss, offsetting the foreign currency translation adjustment of the related net investment that is also recorded in accumulated other comprehensive loss. As of March 31, 2023, the Company had €2,250 million of outstanding long- term debt and approximately €42 million of accrued interest designated as non-derivative hedges of its net investment in certain foreign operations.']


In [12]:
# Get our API key and URL from .env
import os
from dotenv import load_dotenv
load_dotenv()
#api_key = 'mXMwkYwWPgTl2ORNtFEfhCOlkOFQQt6f92hZITvDSKEy'
ibm_cloud_url = 'https://us-south.ml.cloud.ibm.com'
project_id = os.getenv("PROJECT_ID", None)

if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("One or more environment variables are missing!")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }
print("Done getting env variables.")

Done getting env variables.


In [13]:
# LLM that we want to use with watsonx.ai
model_id= "google/flan-ul2"

endpoint= "https://us-south.ml.cloud.ibm.com"

#access_token = ''

#try:
#  access_token = IAMTokenManager(
#    apikey = api_key,
#    url = "https://iam.cloud.ibm.com/identity/token"
#  ).get_token()
#except:
#  print('Issue obtaining access token. Check variables?')

#credentials = { 
#    "url"    : endpoint, 
#    "token" : access_token
#}

# watsonx.ai tuning parameters
gen_params = {
    "DECODING_METHOD" : "greedy",
    "MAX_NEW_TOKENS" : 500,
    "MIN_NEW_TOKENS" : 2,
    "STREAM" : False,
    "TEMPERATURE" : 1.7,
    "TOP_K" : 50,
    "TOP_P" : 1,
    "RANDOM_SEED" : 10
}

model = Model( model_id, creds, gen_params, watsonx_project_id )
print("done")


done


In [22]:
#######################################################################################
prompt_template = """
Article:
###
%s
###

Answer the following question using only information from the article. 
Answer in a complete sentence, with proper capitalization and punctuation. 
If there is no good answer in the article, say "I don't know".

Question: %s
Answer: 
"""

#######################################################################################
def augment( template_in, context_in, query_in ):
    return template_in % ( context_in,  query_in )


#######################################################################################
import json

def generate( model_in, augmented_prompt_in ):
    
    generated_response = model_in.generate( augmented_prompt_in )

    if ( "results" in generated_response ) \
       and ( len( generated_response["results"] ) > 0 ) \
       and ( "generated_text" in generated_response["results"][0] ):
        return generated_response["results"][0]["generated_text"]
    else:
        print( "The model failed to generate an answer" )
        print( "\nDebug info:\n" + json.dumps( generated_response, indent=3 ) )
        return ""


########################################################################################
import re

def query_watsonx(question):
    augmented_prompt = augment( prompt_template, combined_disc_results, question )
    print(augmented_prompt)
    output = generate( model, augmented_prompt )
    if not re.match( r"\S+", output ):
        print( "The model failed to generate an answer")
    print( "\nAnswer:\n" + output )


query_watsonx(discovery_question)

print("\ndone")



Article:
###
['Net Investment Hedge Instruments The Company has designated non-derivative foreign currency denominated long-term debt and the related accrued interest as hedges of its net investment in certain foreign operations. Accordingly, the foreign currency translation of the debt instrument and accrued interest is recorded in accumulated other comprehensive loss, offsetting the foreign currency translation adjustment of the related net investment that is also recorded in accumulated other comprehensive loss. As of March 31, 2023, the Company had €2,250 million of outstanding long- term debt and approximately €42 million of accrued interest designated as non-derivative hedges of its net investment in certain foreign operations.']
###

Answer the following question using only information from the article. 
Answer in a complete sentence, with proper capitalization and punctuation. 
If there is no good answer in the article, say "I don't know".

Question: What is the outstanding lo